In [25]:
#加载数据
import pandas as pd
train = pd.read_csv('Train_DataSet.csv')
train_label = pd.read_csv('Train_DataSet_Label.csv')
test = pd.read_csv('Test_DataSet.csv')
sub = pd.read_csv('submit_example.csv')
sub = sub[['id']]
data = pd.concat([train,test])
data = data.merge(train_label,on='id',how='left')

In [28]:
#简单清洗
import jieba
import re
stop_words = ['的','了','个','这','那','是','在','就','去','有','还','也','与','和','及','或','或者',
              '都','就','和','到','我','上','下','左','右','你','它','他','她']
su='[a-zA-Z0-9’!"#$%&\'()*+,-./:;<=>?@，。?★、…【】《》？“”‘’！[\\]^_`{|}~\s]+'
def split_words(words):
    tmp_words = re.sub(su, "", words)
    cutted_words = ' '.join([i for i in jieba.cut(tmp_words) if i not in stop_words])
    return cutted_words
def clean_w(words):
    tmp_words = re.sub(su, "", words)
    return tmp_words

In [29]:
#合并文本
data['text'] = data['title']+data['content']
data['texta'] = data['text'].astype(str).apply(split_words)
data = data.drop(['title','content'],axis=1)

In [31]:
#训练词向量
from gensim.models import word2vec
import os
def train_wv(data):
    c_w=data['text'].astype(str).apply(clean_w)
    vac=[jieba.lcut(i) for i in c_w]
    model = word2vec.Word2Vec(vac,min_count=1,window=5,size=200)
    model.save('w2v_out.w2v')
    return model

w2v_model = train_wv(data)

/usr/local/anaconda3/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [32]:
#划分数据集
train_data = data[data.label.isin([0,1,2])]
test_data = data[data.id.isin(test['id'].values)]

In [47]:
#使用tokenizer进行转换
from keras.preprocessing.text import Tokenizer
MAX_LEN = 600
tokenizer = Tokenizer()
tokenizer.fit_on_texts(list(data['texta']))
list_tokenized_train = tokenizer.texts_to_sequences(train_data['texta'])
list_tokenized_test = tokenizer.texts_to_sequences(test_data['texta'])
X = pad_sequences(list_tokenized_train, maxlen=MAX_LEN)
X_test = pad_sequences(list_tokenized_test, maxlen=MAX_LEN)

In [35]:
#转换label
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
lb = LabelEncoder()
train_label = lb.fit_transform(train_data["label"].values)
train_label = to_categorical(train_label)

In [49]:
#切分验证集
from sklearn.model_selection import train_test_split
X_train, X_vail, y_train, y_test = train_test_split(X, train_label, test_size=0.1, random_state=42)

In [39]:
#获取词向量矩阵
EMBEDDING_DIM = 200
import numpy as np
word_index = tokenizer.word_index 
embedding_matrix = np.zeros((len(word_index) + 1,EMBEDDING_DIM))
for word,i in word_index.items():
    embedding_matrix[i,:] = w2v_model[word] if word in w2v_model else np.random.rand(EMBEDDING_DIM)

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [52]:
#搭建模型
from keras.layers import *
from keras.models import *
def get_lstm():  
    inp = Input(shape=(MAX_LEN,))
    x = Embedding(WORD_IDX_LEN + 1, EMBEDDING_DIM, weights=[embedding_matrix], trainable=False)(inp)
    x = LSTM(128,return_sequences=True,name='lstm_layer')(x)

    x = GlobalMaxPool1D()(x)
    x = Dense(512,activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(512,activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(256,activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(3,activation='softmax')(x)
    lstm_model = Model(inputs=inp,outputs=x)
#     lstm_model = multi_gpu_model(lstm_model,  gpus=3)
    lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return lstm_model

In [53]:
#----------训练
from keras.callbacks import *
file_path= "word_weight_ccf.hdf"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)
early_stopping =EarlyStopping(monitor='val_loss', patience=3)
WORD_IDX_LEN = len(word_index)

model = get_lstm()

model.fit(X_train,y_train,batch_size=32,epochs=10,verbose=1,
                               validation_data=(X_vail,y_test),
                               callbacks=[checkpoint,early_stopping])
model.load_weights(file_path)

Train on 6606 samples, validate on 734 samples
Epoch 1/10
6606/6606 [==============================] - 130s 20ms/step - loss: 0.7328 - acc: 0.6820 - val_loss: 0.5739 - val_acc: 0.7520

Epoch 00001: val_loss improved from inf to 0.57386, saving model to word_weight_ccf.hdf
Epoch 2/10
6606/6606 [==============================] - 128s 19ms/step - loss: 0.6061 - acc: 0.7401 - val_loss: 0.5747 - val_acc: 0.7507

Epoch 00002: val_loss did not improve from 0.57386
Epoch 3/10
6606/6606 [==============================] - 127s 19ms/step - loss: 0.5573 - acc: 0.7579 - val_loss: 0.5324 - val_acc: 0.7616

Epoch 00003: val_loss improved from 0.57386 to 0.53237, saving model to word_weight_ccf.hdf
Epoch 4/10
6606/6606 [==============================] - 126s 19ms/step - loss: 0.4963 - acc: 0.7838 - val_loss: 0.5498 - val_acc: 0.7548

Epoch 00004: val_loss did not improve from 0.53237
Epoch 5/10
6606/6606 [==============================] - 128s 19ms/step - loss: 0.4578 - acc: 0.8043 - val_loss: 0.6212 

In [55]:
#提交
def z(x):
    if x==0:
        return '0'
    elif x==1:
        return '1'
    else:
        return '2'
    
pred_word= lb.inverse_transform(np.argmax(model.predict(X_test), 1)).reshape(-1,1)
test_data['label'] = [i[0] for i in pred_word]
sub = sub.merge(test_data[['id','label']],on='id',how='left')
sub['label'] = sub['label'].apply(z)
sub.to_csv('res_base.csv',index=None)

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
